In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import json
import matplotlib.pyplot as plt
import shutil

In [2]:
import os
print(os.getcwd())

d:\Documents\schoolwork-files\archive\PhD_Research\2022\ComputerVision\screws_project


In [3]:
with open("./mvtec_screws_v1.0/mvtec_screws.json", "r") as f:
    annots = json.load(f)

annotations = annots["annotations"]
categories = annots["categories"]
images = annots["images"]
print("Objects "+str(len(annotations)))
print("Categories "+str(len(categories)))
print("Images "+str(len(images)))

Objects 4427
Categories 13
Images 384


In [4]:
images_dir = './mvtec_screws_v1.0/images'
tfrecords_dir = "tfrecords"

In [5]:
file_count = sum([len(files) for r, d, files in os.walk(images_dir)])
num_samples = len(images)
num_tfrecords = file_count // num_samples

In [10]:
def _image_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(value).numpy()])
    )

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def create_example(image, path, example):
    feature = {
        "image": _image_feature(image),
        "area": _float_feature(example["area"]),
        
        #Originally annotated as bounding boxes. Each box contains 5 parameters (row, col, width, height, phi)
        "row": _float_feature(example["bbox"][0]),
        "col": _float_feature(example["bbox"][1]),
        "width": _float_feature(example["bbox"][2]),
        "height": _float_feature(example["bbox"][3]),
        "phi": _float_feature(example["bbox"][4]),

        "category_id": _int64_feature(example["category_id"]),
        "id": _int64_feature(example["id"]),
        "image_id": _int64_feature(example["image_id"]),
        "is_crowd": _int64_feature(example["is_crowd"]),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))


def parse_tfrecord_fn(example):
    feature_description = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "area": tf.io.FixedLenFeature([], tf.float32),

        #Originally annotated as bounding boxes. Each box contains 5 parameters (row, col, width, height, phi)
        "row": tf.io.FixedLenFeature([], tf.float32),
        "col": tf.io.FixedLenFeature([], tf.float32),
        "width": tf.io.FixedLenFeature([], tf.float32),
        "height": tf.io.FixedLenFeature([], tf.float32),
        "phi": tf.io.FixedLenFeature([], tf.float32),

        "category_id": tf.io.FixedLenFeature([], tf.int64),
        "id": tf.io.FixedLenFeature([], tf.int64),
        "image_id": tf.io.FixedLenFeature([], tf.int64),
        "is_crowd": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    example["image"] = tf.io.decode_jpeg(example["image"], channels=3)
    return example

In [11]:
if not os.path.exists(tfrecords_dir):
    os.makedirs(tfrecords_dir)  # creating TFRecords output folder
for tfrec_num in range(num_tfrecords):
    samples = annotations[(tfrec_num * num_samples) : ((tfrec_num + 1) * num_samples)]

    with tf.io.TFRecordWriter(
        tfrecords_dir + "/file_%.2i-%i.tfrec" % (tfrec_num, len(samples))
    ) as writer:
        for sample in samples:
            for image in images:
                if sample['image_id'] == image['id']:
                    image_path = os.path.join(images_dir, image['file_name'])
            image = tf.io.decode_jpeg(tf.io.read_file(image_path))
            example = create_example(image, image_path, sample)
            writer.write(example.SerializeToString())